# QuantFredom Video 1 - Converting TradingView RSI strategy to Python




In [42]:
# YT Video - https://www.youtube.com/watch?v=-PepEj8BM6g
# import libs
import os
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from numba import njit
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('MFX_KEY')
api_secret = os.getenv('MFX_SECRET')

mufex_main = Mufex(
  api_key=api_key,
  secret_key=api_secret,
  use_test_net=False
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Testing testing
y = np.arange(10)
print(y)
y = np.roll(y, -1)
y

[0 1 2 3 4 5 6 7 8 9]


array([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [43]:
# Fetch candles
candles = mufex_main.get_candles(symbol='BTCUSDT', timeframe='5m', candles_to_dl=300)
# print(candles[0])

# Get Prices
prices = candles[:, 4]
# print(prices)
period = 14

In [54]:
# Prior price 
prices_shift = np.roll(prices, 1)
prices_shift[0] = np.nan
print(f'Current Price: {prices[-1]}')
print(f'Previous Price: {prices_shift[-1]}') 

# Pct Change
pchg = (prices - prices_shift) / prices_shift
print(f'Change: {pchg[-1]}')
alpha = 1 / period
gain = np.where(pchg > 0, pchg, 0)
avg_gain = np.full_like(gain, np.nan)

loss = np.where(pchg < 0, abs(pchg), 0)
avg_loss = np.full_like(loss, np.nan)

avg_gain[period] = gain[1 : period + 1].mean()
avg_loss[period] = loss[1 : period + 1].mean()

Current Price: 68428.4
Previous Price: 68528.7
Change: -0.0014636203517650694


In [19]:
# RSI Indicator
def qf_calc_rsi(prices: np.array, period: int):
  prices_shift = np.roll(prices, 1)
  prices_shift[0] = np.nan
  pchg = (prices - prices_shift) / prices_shift

  alpha = 1 / period
  gain = np.where(pchg > 0, pchg, 0)
  avg_gain = np.full_like(gain, np.nan)

  loss = np.where(pchg < 0, abs(pchg), 0)
  avg_loss = np.full_like(loss, np.nan)

  avg_gain[period] = gain[1 : period + 1].mean()
  avg_loss[period] = loss[1 : period + 1].mean()

  for i in range(period, + 1, gain.size):
    avg_gain[i] = alpha * gain[i] + (1 - alpha) * avg_gain[i - 1]
    avg_loss[i] = alpha * loss[i] + (1 - alpha) * avg_loss[i - 1]

  rs = avg_gain / avg_loss

  rsi = 100 - (100 / (i + rs))
  return rsi

